In [3]:
#Import data of stocks in S&P 500 present from 1999 to 2019
import yfinance as yf
import numpy as np
import pandas as pd
import nasdaqdatalink
import requests
from bs4 import BeautifulSoup

NASDAQ_DATA_LINK_API_KEY = "TqyaGpQ_6SKSx38euQUU"
# Get the S&P 500 constituents
sp500_constituents_url = "https://en.wikipedia.org/wiki/List_of_S&P_500_companies"
sp500_constituents_df = pd.read_html(sp500_constituents_url)[0]
#print(sp500_constituents_df["Symbol"])
symbols = sp500_constituents_df["Symbol"]


In [11]:
# Filter out the stocks in ticker list that are not present in S&P 500 from 1999 to 2019
# Get the data of the stocks in S&P 500 from 1999 to 2019
print(f'Original Length: {len(symbols)}')
#symbols = ['AAPL']
tickers = []
for symbol in symbols:
    try: 
        data = yf.download(symbol, start="1999-01-01", end="2019-12-31", progress=False)
        #print(data)
        #data.index = data['Date']
        #print(data.head())
        #print(data.index[0].year)
        #print(data.index[0].month)
        #print(data.index[0].day)
        #print(data.index[-1].year)
        #print(data.index[-1].month)
        #print(data.index[-1].day)
        
        if [data.index[0].year, data.index[0].month, data.index[0].day] == [1999, 1, 4] and [data.index[-1].year, data.index[-1].month, data.index[-1].day] == [2019, 12, 30]:
            #print("Yes")
            tickers.append(symbol)
            
            # Add Returns column
            data['Returns'] = data['Adj Close'].pct_change() + 1
            data.dropna(inplace=True)
            data.to_csv(f"data/{symbol}.csv")
        
    except:
        continue
        
print(f'Length after filtering: {len(tickers)}')  
    

Original Length: 502



1 Failed download:
['ABNB']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1999-01-01 -> 2019-12-31)')

1 Failed download:
['CARR']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['CEG']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['GEHC']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['OGN']: Exception("%ticker%: Data doesn't exist for startDate = 915166800, endDate = 1577768400")

1 Failed download:
['OTIS']: Exception("%ticker%: Data doe

Length after filtering: 348


In [12]:
# Download S&P 500 data from 1999 to 2019 as baseline data
data = yf.download("^GSPC", start="1999-01-01", end="2019-12-31", interval="1d")
data['Market Returns'] = data['Adj Close'].pct_change()
data.dropna(inplace=True)
data.to_csv(f"data/GSPC.csv")

[*********************100%%**********************]  1 of 1 completed


In [14]:

import os
import pandas as pd
import statsmodels.api as sm

DATA_DIR = (os.getcwd() + "/data/")
import requests 
from bs4 import BeautifulSoup
start_date = '1999-01-01'
end_date = "2019-12-31"
# Get fundamental data for each stock in the ticker and append to the dataframe

def get_all_symbols():
    return [v.strip('.csv') for v in os.listdir(DATA_DIR)]

tickers = get_all_symbols()
#tickers = ['AAPL']
market_data = pd.read_csv(f"data/GSPC.csv", index_col = 0)


for ticker in tickers:
    ticker_data = pd.read_csv(f"data/{ticker}.csv", index_col=0)
    print(f"Current Ticker: {ticker}")
    #Skip if columns exists
    if ticker == 'GSPC' or 'Short Term Reversal' in ticker_data.columns.tolist():
        continue
    #Short Term Reversal, Stock Momentum, Long Term Reversal
    st_period = 5
    lt_period = 20
    ticker_data['Short Term Reversal'] = ticker_data['Close'].pct_change(
        periods=st_period)
    
    ticker_data['Stock Momentum'] = (
        ticker_data['Close'] / ticker_data['Close'].shift(st_period)) - 1
    
    ticker_data['Long Term Reversal'] = ticker_data['Close'].pct_change(periods=lt_period)
    

    # Calculate Market Beta for each stock    
    market_betas = []
    for date in ticker_data.index:
        # Filter market data for the specific date
        market_data_for_date = market_data[market_data.index <= date]

        # Calculate Market Beta for that date
        X = sm.add_constant(market_data_for_date['Market Returns'])
        y = ticker_data.loc[ticker_data.index <= date, 'Returns']
        model = sm.OLS(y, X).fit()
        beta = model.params['Market Returns']
        market_betas.append(beta)
        
    # Add Market Beta as a new column in the stock data
    ticker_data['Market_Beta'] = market_betas
    
    
    # Calculating Turnover, Turnover Volatility, and Total Return Volatility
    
    # Taking Volume as Turnover
    ticker_data['Turnover Volatility'] = ticker_data['Volume'].rolling(window=lt_period).std()
    
    # Add Dividends
    ticker_historical = yf.Ticker(ticker)
    historical = ticker_historical.history(period='1mo', start=start_date, end=end_date)
    
    ticker_dividends = ticker_historical.dividends
    ticker_data['Dividends'] = ticker_dividends
    ticker_data['Dividends'].fillna(0.0, inplace=True)
    
    ticker_data['Total Returns'] = ticker_data['Close'].pct_change() + (ticker_data['Dividends'] / ticker_data['Close'])
    ticker_data['Total Return Volatility'] = ticker_data['Total Returns'].rolling(window=lt_period).std()
        
    ticker_data.dropna(inplace=True)
    ticker_data.to_csv(f"data/{ticker}.csv")
    #print(ticker_data.head())
        
        # Get the fundamental data
        #data_url = f"https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}"
        #data_response = requests.get(data_url)
        #data = data_response.content.decode()
        #soup = BeautifulSoup(data, 'html.parser')
        #print(soup.prettify())
        #print(soup.find_all("div", {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"}))
    #print(ticker_data)
    #current_year, current_month = ticker
    #data_url = f"https://finance.yahoo.com/quote/{symbol}?p={symbol}"
    #data_response = requests.get(data_url)

    #data = data_response.content.decode()
    #market_cap_start_index = data.find("Market Cap:")
    #market_cap_end_index = data.find(
    #    "</span>", market_cap_start_index)
    #market_cap = data[market_cap_start_index +
    #                             12:market_cap_end_index].strip()

    


Current Ticker: AAPL
Current Ticker: ABT
Current Ticker: ACGL
Current Ticker: ADBE
Current Ticker: ADI
Current Ticker: ADM
Current Ticker: ADP
Current Ticker: ADSK
Current Ticker: AEE
Current Ticker: AEP
Current Ticker: AES
Current Ticker: AFL
Current Ticker: AIG
Current Ticker: AJG
Current Ticker: ALB
Current Ticker: ALK
Current Ticker: ALL
Current Ticker: AMAT
Current Ticker: AMD
Current Ticker: AME
Current Ticker: AMGN
Current Ticker: AMT
Current Ticker: AMZN
Current Ticker: ANSS
Current Ticker: AON
Current Ticker: AOS
Current Ticker: APA
Current Ticker: APD
Current Ticker: APH
Current Ticker: ARE
Current Ticker: ATO
Current Ticker: AVB
Current Ticker: AVY
Current Ticker: AXP
Current Ticker: AZO
Current Ticker: BA
Current Ticker: BAC
Current Ticker: BALL
Current Ticker: BAX
Current Ticker: BBWI
Current Ticker: BBY
Current Ticker: BDX
Current Ticker: BEN
Current Ticker: BIIB
Current Ticker: BIO
Current Ticker: BK
Current Ticker: BKR
Current Ticker: BMY
Current Ticker: BRO
Current Tic

In [6]:
# Add in RSI, df['SMA200'] = TA.SMA(df, 200)
import requests
from bs4 import BeautifulSoup
import pandas_ta as ta
import os
from finta import TA
import pandas as pd
import statsmodels.api as sm

DATA_DIR = (os.getcwd() + "/data/")
start_date = '1999-01-01'
end_date = "2019-12-31"
# Get fundamental data for each stock in the ticker and append to the dataframe


def get_all_symbols():
    return [v.strip('.csv') for v in os.listdir(DATA_DIR)]


tickers = get_all_symbols()
#tickers = ['AAPL']
market_data = pd.read_csv(f"data/GSPC.csv", index_col=0)


for ticker in tickers:
    df = pd.read_csv(f"data/{ticker}.csv", index_col=0)
    print(f"Current Ticker: {ticker}")
    # Skip if columns exists
    if ticker == 'GSPC':
        continue
    df = df.assign(
        SMA_5=df['Close'].rolling(5).mean(),
        SMA_20=df['Close'].rolling(20).mean(),
        SMA_50=df['Close'].rolling(50).mean(),
        SMA_252=df['Close'].rolling(252).mean(),
        adv20=df["Volume"].rolling(20).mean(),
        VWAP=(df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum()
    )
    
    # Add in log returns
    df['log_returns'] = np.log(df['Close']) - np.log(df['Close'].shift(1))

    # add in monthly volatility
    df['volatility_30'] = df['log_returns'].rolling(30).std() * np.sqrt(30)
    df['volatility_60'] = df['log_returns'].rolling(60).std() * np.sqrt(60)
    df['annual_volatility'] = df['log_returns'].rolling(
        252).std() * np.sqrt(252)
    
    df["RSI(2)"] = ta.rsi(df['Close'], length=2)
    df["RSI(7)"] = ta.rsi(df['Close'], length=7)
    df["RSI(14)"] = ta.rsi(df['Close'], length=14)
    df["CCI(30)"] = ta.cci(close=df['Close'],
                           length=30, high=df["High"], low=df["Low"])
    df["CCI(50)"] = ta.cci(close=df['Close'],
                           length=50, high=df["High"], low=df["Low"])
    df["CCI(100)"] = ta.cci(close=df['Close'],
                            length=100, high=df["High"], low=df["Low"])
    df['BBWidth'] = TA.BBWIDTH(df)
    df['Williams'] = TA.WILLIAMS(df)


    df.dropna(inplace=True)
    print(df)
    df.to_csv(f"data/{ticker}.csv")


Current Ticker: AAPL
                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2000-02-02   0.899554   0.911830   0.866071   0.882254   0.748854  464195200   
2000-02-03   0.895647   0.930804   0.895089   0.922433   0.782958  475193600   
2000-02-04   0.928013   0.982143   0.925223   0.964286   0.818483  425320000   
2000-02-07   0.964286   1.020089   0.945871   1.018415   0.864427  441067200   
2000-02-08   1.017857   1.036830   0.993304   1.025670   0.870585  408643200   
...               ...        ...        ...        ...        ...        ...   
2019-12-23  70.132500  71.062500  70.092499  71.000000  69.261597   98572000   
2019-12-24  71.172501  71.222504  70.730003  71.067497  69.327446   48478800   
2019-12-26  71.205002  72.495003  71.175003  72.477501  70.702927   93121200   
2019-12-27  72.779999  73.492500  72.029999  72.449997  70.676102  146266000   
2019-12-30  72.3649

In [2]:
# Generating Close data 
import os

DATA_DIR = (os.getcwd() + "/data/")
PREDICTION_DIR = (os.getcwd() + "/predictions_new/")

def get_all_symbols():
    return [v.strip('.csv') for v in os.listdir(DATA_DIR)]

def get_current_predictions():
    return [v.strip('_predictions.csv') for v in os.listdir(PREDICTION_DIR)]

tickers = get_current_predictions()
#tickers = ['AAPL', 'MSFT']
result_1year = pd.DataFrame()
result_2year = pd.DataFrame()
result_3year = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv(f"data/{ticker}.csv", index_col=0)
    mask1 = (df.index>= '2013-01-01') & (df.index <= '2019-12-31')
    mask2 = (df.index>= '2012-01-01') & (df.index <= '2019-12-31')
    mask3 = (df.index>= '2011-01-01') & (df.index <= '2019-12-31')
    

    df1 = df.loc[mask1]
    df1 = df1[['Close']]
    df1 = df1.rename(columns={'Close': ticker})
    
    df2 = df.loc[mask2]
    df2 = df2[['Close']]
    df2 = df2.rename(columns={'Close': ticker})
    
    df3 = df.loc[mask3]
    df3 = df3[['Close']]
    df3 = df3.rename(columns={'Close': ticker})

    result_1year = pd.concat([result_1year, df1], axis=1)
    result_2year = pd.concat([result_2year, df2], axis=1)
    result_3year = pd.concat([result_3year, df3], axis=1)
    


NameError: name 'pd' is not defined

In [17]:
# Save result 
result_1year.to_csv('close_data_1year.csv')
result_2year.to_csv('close_data_2year.csv')
result_3year.to_csv('close_data_3year.csv')

In [ ]:
# Generate Predicted Historical Return Data 
output = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv(f"predictions_new/{ticker}_predictions.csv", index_col=0)
    df = df[['Next Month']]
    df = df.rename(columns={'Close': ticker})
    output = pd.concat([output, df], axis=1)

In [17]:
# Generate returns data from close prices
close_data = pd.read_csv('close_data_1year.csv', index_col=0)
# Generate returns data from close prices, resampled to M
close_data.index = pd.to_datetime(close_data.index)
close_data = close_data.resample('M').last()
close_data = close_data.pct_change()
close_data = close_data.dropna()
mask = (close_data.index>= '2013-12-01') & (close_data.index <= '2019-11-01')
close_data = close_data.loc[mask]
close_data['Date for next'] = close_data.index + pd.DateOffset(days=1)
close_data = close_data.set_index('Date for next')
close_data
close_data.to_csv('returns_data.csv')